In [1]:
import os
import csv
import json
import pandas as pd
from collections import Counter, defaultdict
from sample_descriptions import get_desc_200, get_desc_300

``` 
0 Correct
--------------
Moves to change correct structure but
1 builds over existing structure/in line with structure but not touching
2 adds to structure but adds too many
3 just picks and places one block (or two blocks)
4 makes pick moves instead of place (NB sometimes these are picks of blocks that aren’t there)
5 makes incoherent builds
(e.g. adds a tower instead of a row, builds a new structure, starts far away etc,)
--------
6 Moves to change wrong structure
7 Moves to change both (all) structures and changes correct one correctly
8 Moves to change both (all) structures and neither is correctly changed
9 Just starts building somewhere else on board
10 Correct placement, wrong color
```

```

200
Counter({'TT': 56, 'TB': 56, 'RB': 48, 'RT': 40})
Counter({'different': 167, 'two same': 33})

300
Counter({'TBB': 48, 'RBB': 38, 'RTB': 37, 'TTT': 37, 'RTT': 37, 'TTB': 36, 'TBT': 35, 'RBT': 32})
Counter({'different': 165, 'two same': 125, 'three same': 10})

```

In [40]:
home=%pwd
# csv_path = home + '/Corr-synth-Long.csv'
csv_path = home + '/Corr-synth-Short.csv'

In [41]:
df = pd.read_csv(csv_path)

In [42]:
df.shape

(200, 9)

In [43]:
with open('/home/kate/minecraft_utils/synthetic/corrections/synthetic_corrections_short_check_freeze.txt') as f:
    samples = f.read().split('\n')

In [44]:
# colors, shapes = get_desc_300(samples)
colors, shapes = get_desc_200(samples)

In [45]:
df = df.assign(COLORS=colors)

In [46]:
df = df.assign(SHAPES=shapes)

In [47]:
merge_row = []
for i, s in enumerate(colors):
    merge_row.append(shapes[i] + s)

In [48]:
df = df.assign(MERGE=merge_row)

In [49]:
df.head(10)

,SAMPLE,Type,LLAMIPA-P,BASELINE,STRUCT,NO_STRUCT,NARR_ARC,FULL_STRUCT,FULL_NO_STRUCT,COLORS,SHAPES,MERGE
0,1,2,1,8,0,1,4,0,0,D,TT,TTD
1,2,2,2,0,0,1,8,0,0,D,RB,RBD
2,3,2,1,6,0,0,0,4,6,2S,TT,TT2S
3,4,2,1,6,0,0,6,4,4,2S,TT,TT2S
4,5,2,2,0,7,8,0,3,0,2S,TB,TB2S
5,6,2,2,1,1,1,1,5,4,D,RB,RBD
6,7,2,1,0,2,1,4,0,0,D,RT,RTD
7,8,2,2,0,0,1,0,0,0,D,TB,TBD
8,9,2,2,0,0,1,0,0,0,D,TB,TBD
9,10,2,2,0,0,0,0,0,0,D,RB,RBD


In [50]:
#LLamipa scores
# df[(df['LLAMIPA-P'] == 2)&(df['Type'] == 2)]
Counter(df[df['LLAMIPA-P'] != df['Type']]['Type'].tolist())

Counter({2: 60})

##### what are samples with llamipa mistakes?

In [78]:
Counter(df[df['LLAMIPA-P'] != df['Type']]['COLORS'].tolist())
#Counter(df.loc[df['Type']== 3]['COLORS'].tolist())
# Counter(df['COLORS'].tolist())

Counter({'2S': 16, 'D': 13, '3S': 3})

In [77]:
Counter(df[df['LLAMIPA-P'] != df['Type']]['SHAPES'].tolist())
#Counter(df.loc[df['Type'] == 3]['SHAPES'].tolist())
#Counter(df['SHAPES'].tolist())

Counter({'TTT': 12,
         'RTT': 6,
         'TTB': 4,
         'TBT': 3,
         'RBT': 3,
         'RTB': 2,
         'TBB': 2})

In [111]:
Counter(df['MERGE'].tolist())
#len(df['MERGE'].tolist())
#Counter(df.loc[df['Type']== 3]['MERGE'].tolist())
#len(df.loc[df['Type']== 3]['MERGE'].tolist())
#df[(df['Type'] == 2)&(df['SHAPES'].isin(['RTB', 'RBT']))&(df['COLORS']).isin(['2S', '3S'])].shape[0]
#df[(df['Type'] == 2)&(df['SHAPES'].isin(['RTB', 'RBT']))&(df['COLORS']).isin(['D'])].shape[0]
#df[(df['Type'] == 2)&(~df['SHAPES'].isin(['RTB', 'RBT']))&(df['COLORS']).isin(['D'])].shape[0]

Counter({'TTD': 48,
         'TBD': 46,
         'RBD': 42,
         'RTD': 31,
         'TB2S': 10,
         'RT2S': 9,
         'TT2S': 8,
         'RB2S': 6})

In [14]:
#only shape ambiguous 
#df[(df['Type'] == 3)&(~df['SHAPES'].isin(['RTB', 'RBT']))&(df['COLORS'].isin(['D'])&(df['LLAMIPA-P'] != df['Type']))].shape[0]
#only color ambiguous
#df[(df['Type'] == 3)&(df['SHAPES'].isin(['RTB', 'RBT']))&(~df['COLORS'].isin(['D'])&(df['LLAMIPA-P'] != df['Type']))].shape[0]
#no ambiguity and llamipa wrong
#df[(df['Type'] == 3)&(df['SHAPES'].isin(['RTB', 'RBT']))&(df['COLORS'].isin(['D'])&(df['LLAMIPA-P'] != df['Type']))].shape[0]
#both ambiguous 
df[(df['Type'] == 3)&(~df['SHAPES'].isin(['RTB', 'RBT']))&(~df['COLORS'].isin(['D'])&(df['LLAMIPA-P'] != df['Type']))].shape[0]

14

##### checking the ambiguities associated with wrong shapes

In [73]:
#both ambiguous
#Counter(df[(df['NO_STRUCT'].isin([6,7,8,9]))&(~df['SHAPES'].isin(['RTB', 'RBT']))&(~df['COLORS'].isin(['D']))]['Type'])
Counter(df[(df['NARR_ARC'].isin([6,7,8,9]))&(~df['SHAPES'].isin(['RT', 'RB', 'TB']))&(~df['COLORS'].isin(['D']))]['Type'])

Counter({2: 1})

In [74]:
#no ambiguity
# Counter(df[(df['NO_STRUCT'].isin([6,7,8,9]))&(df['SHAPES'].isin(['RTB', 'RBT']))&(df['COLORS'].isin(['D']))]['Type'])
Counter(df[(df['NARR_ARC'].isin([6,7,8,9]))&(df['SHAPES'].isin(['RT', 'RB', 'TB']))&(df['COLORS'].isin(['D']))]['Type'])

Counter({2: 14, 1: 1})

In [75]:
#only color ambiguous
#Counter(df[(df['NO_STRUCT'].isin([6,7,8,9]))&(df['SHAPES'].isin(['RTB', 'RBT']))&(~df['COLORS'].isin(['D']))]['Type'])
Counter(df[(df['NARR_ARC'].isin([6,7,8,9]))&(df['SHAPES'].isin(['RT', 'RB', 'TB']))&(~df['COLORS'].isin(['D']))]['Type'])

Counter({2: 7})

In [76]:
#only shape ambiguous
# Counter(df[(df['NO_STRUCT'].isin([6,7,8,9]))&(~df['SHAPES'].isin(['RTB', 'RBT']))&(df['COLORS'].isin(['D']))]['Type'])
Counter(df[(df['NARR_ARC'].isin([6,7,8,9]))&(~df['SHAPES'].isin(['RT', 'RB', 'TB']))&(df['COLORS'].isin(['D']))]['Type'])

Counter()

In [134]:
#df[(df['Type'] == 2)&(~df['SHAPES'].isin(['RT', 'RB', 'TB']))&(df['COLORS']).isin(['D'])].shape[0]
#df[(df['Type'] == 2)&(df['SHAPES'].isin(['RT', 'RB', 'TB']))&(~df['COLORS'].isin(['D']))].shape[0]
df[(df['Type'] == 2)&(df['SHAPES'].isin(['RT', 'RB', 'TB']))&(df['COLORS'].isin(['D']))].shape[0]

60

In [141]:
#only shape ambiguous 
#df[(df['Type'] == 2)&(~df['SHAPES'].isin(['RT', 'RB', 'TB']))&(df['COLORS'].isin(['D'])&(df['LLAMIPA-P'] != df['Type']))].shape[0]
#only color ambiguous
#df[(df['Type'] == 2)&(df['SHAPES'].isin(['RT', 'RB', 'TB']))&(~df['COLORS'].isin(['D'])&(df['LLAMIPA-P'] != df['Type']))].shape[0]
#no ambiguity and llamipa wrong
#df[(df['Type'] == 2)&(df['SHAPES'].isin(['RT', 'RB', 'TB']))&(df['COLORS'].isin(['D'])&(df['LLAMIPA-P'] != df['Type']))].shape[0]
#both ambiguous 
df[(df['Type'] == 2)&(~df['SHAPES'].isin(['RT', 'RB', 'TB']))&(~df['COLORS'].isin(['D'])&(df['LLAMIPA-P'] != df['Type']))].shape[0]

3

In [80]:
#Counter(df[df['LLAMIPA-P'] != df['Type']]['MERGE'].tolist())
len(df[df['LLAMIPA-P'] != df['Type']]['MERGE'].tolist())


32

##### Other model scores

In [25]:
#Baseline scores 
Counter(df.loc[df['BASELINE'].isin([0]), 'Type'].tolist())

Counter({1: 100, 2: 73, 3: 64})

In [26]:
#Narr scores 
Counter(df.loc[df['NARR_ARC'].isin([0]), 'Type'].tolist())

Counter({1: 97, 2: 82, 3: 38})

In [24]:
#Nebulipa Struct
Counter(df.loc[df['FULL_STRUCT'].isin([0]), 'Type'].tolist())

Counter({1: 100, 2: 75, 3: 66})

In [27]:
#Nebulipa NO Struct
Counter(df.loc[df['FULL_NO_STRUCT'].isin([0]), 'Type'].tolist())

Counter({1: 98, 2: 57, 3: 46})

##### for those that were incorrect, how many were attached to the wrong structure?

In [85]:
Counter(df.loc[df['BASELINE']!= 0]['Type'].tolist()), Counter(df.loc[df['BASELINE'].isin([6,7,8,9]), 'Type'].tolist())


(Counter({2: 32, 1: 1}), Counter({2: 10}))

In [86]:
Counter(df.loc[df['NARR_ARC']!= 0]['Type'].tolist()), Counter(df.loc[df['NARR_ARC'].isin([6,7,8,9]), 'Type'].tolist())

(Counter({2: 61, 1: 2}), Counter({2: 22, 1: 1}))

In [37]:
Counter(df.loc[df['FULL_STRUCT']!= 0]['Type'].tolist()), Counter(df.loc[df['FULL_STRUCT'].isin([6,7,8,9]), 'Type'].tolist())

(Counter({3: 34, 2: 25}), Counter({2: 5, 3: 2}))

In [39]:
Counter(df.loc[df['FULL_NO_STRUCT']!= 0]['Type'].tolist()), Counter(df.loc[df['FULL_NO_STRUCT'].isin([6,7,8,9]), 'Type'].tolist())

(Counter({3: 54, 2: 43, 1: 2}), Counter({2: 8, 3: 3}))

##### for those that were incorrect, how many were also incorrectly predicted by llamipa?

In [30]:
Counter(df[(df['BASELINE'].isin([6,7,8,9]))&(df['LLAMIPA-P'] != df['Type'])]['Type'].tolist())

Counter({3: 3})

In [31]:
Counter(df[(df['NARR_ARC'].isin([6,7,8,9]))&(df['LLAMIPA-P'] != df['Type'])]['Type'].tolist())

Counter({3: 8})

In [32]:
Counter(df[(df['STRUCT'].isin([6,7,8,9]))&(df['LLAMIPA-P'] != df['Type'])]['Type'].tolist())

Counter()

In [33]:
Counter(df[(df['NO_STRUCT'].isin([6,7,8,9]))&(df['LLAMIPA-P'] != df['Type'])]['Type'].tolist())

Counter({3: 6})

In [34]:
Counter(df[(df['FULL_STRUCT'].isin([6,7,8,9]))&(df['LLAMIPA-P'] != df['Type'])]['Type'].tolist())

Counter({3: 1})

In [38]:
Counter(df[(df['FULL_NO_STRUCT'].isin([6,7,8,9]))&(df['LLAMIPA-P'] != df['Type'])]['Type'].tolist())

Counter({3: 1})

In [ ]:
#Counter(df['Loc_corr'].tolist())

In [19]:
#which samples didn't get color right? Only when the moves were botched
df.loc[df['Color'] == 0]

,Shape,Color,Shape_corr,Size_corr,Loc_corr,Orient_corr,In_bounds,Full_struct_corr
26,square,0,1,1,0.0,NaN,1,0
53,square,0,1,1,0.0,NaN,1,0
54,square,0,0,0,0.0,NaN,1,0
55,square,0,0,0,0.0,NaN,1,0


In [9]:
#what the breakdown of correct and incorrect shapes?
#Correct:
Counter(df.loc[df['Shape_corr'] == 1, 'Shape'].tolist())

Counter({'tower': 168,
         'row': 166,
         'rectangle': 83,
         'square': 77,
         'diagonal': 76,
         'cube': 20})

In [10]:
#what the breakdown of correct and incorrect shapes?
#INorrect:
Counter(df.loc[df['Shape_corr'] == 0, 'Shape'].tolist())

Counter({'diagonal': 92,
         'square': 75,
         'diamond': 52,
         'rectangle': 46,
         'cube': 4,
         'row': 2})

In [ ]:
#Incorrect:
Counter(df.loc[df['Shape_corr'].isin([0,1]), 'Shape'].tolist())

In [12]:
#what the breakdown of correct and incorrect sizes?
#Correct:
Counter(df.loc[df['Size_corr'] == 1, 'Shape'].tolist())

Counter({'tower': 168,
         'row': 164,
         'square': 77,
         'diagonal': 73,
         'rectangle': 18,
         'cube': 14})

In [ ]:
#what the breakdown of correct and incorrect locations?
#Correct:
Counter(df.loc[df['Loc_corr'] == 1, 'Shape'].tolist())

In [ ]:
Counter(df[(df['Size_corr'] == 0)&(df['Shape_corr'] == 1)]['Shape'].tolist())

In [13]:
Counter(df[(df['Loc_corr'].isin([1,0]))&(df['Shape_corr'] == 1)]['Shape'].tolist())

Counter({'tower': 142,
         'row': 135,
         'rectangle': 77,
         'diagonal': 66,
         'square': 58,
         'cube': 15})

In [14]:
Counter(df[(df['Loc_corr'] == 1)&(df['Shape_corr'] == 1)]['Shape'].tolist())

Counter({'row': 70,
         'tower': 64,
         'rectangle': 41,
         'diagonal': 14,
         'square': 7,
         'cube': 4})

In [ ]:
#Shape corr and location specified
Counter(df[(df['Shape_corr'] == 1)&(df['Loc_corr'].isin([1,0]))]['Shape'].tolist())

In [15]:
#Shape corr and is in bounds
Counter(df[(df['Shape_corr'] == 1)&(df['In_bounds'] == 0)&(df['Loc_corr'].isin([0, 1]))]['Shape'].tolist())

Counter({'rectangle': 2, 'row': 2, 'diagonal': 2})

In [ ]:
#Shape corr and is in bounds
Counter(df[(df['Shape_corr'] == 1)&(df['In_bounds'] == 0)&(df['Loc_corr'].isin([1]))]['Shape'].tolist())

In [16]:
Counter(df[(df['Orient_corr'].isin([1,0]))&(df['Shape_corr'] == 1)]['Shape'].tolist())

Counter({'rectangle': 79, 'square': 72})

In [17]:
Counter(df[(df['Orient_corr'].isin([1]))&(df['Shape_corr'] == 1)]['Shape'].tolist())

Counter({'rectangle': 75, 'square': 68})

In [ ]:
#get orientation types
indices = df.index[(df['Orient_corr'].isin([1]))&(df['Shape_corr'] == 1)].tolist()

In [ ]:
json_path = home +  '/llama_aug_synth_function_output_v2.json'

with open(json_path, 'r') as j:
    jfile = json.load(j)
    samples = jfile

In [ ]:
orient_dict = defaultdict(list)

for sample in samples:
    shape = sample['shape'] 
    if sample['index'] in indices:
        orient_dict[shape].append(sample['orient'])
        

In [ ]:
Counter(orient_dict['square'])

In [ ]:
Counter(orient_dict['diamond'])

#### for each incorrect square, get the instruction size, then check if square

In [ ]:
import functions as fn

In [ ]:
rec_squares = [['index', 'inst_size', 'rec_size']]

In [ ]:
#get squares that are wrong because they are rectangles -- maybe try "is rectangle" in these cases
indices = df.index[df['Shape'].isin(['square'])& df['Shape_corr'].isin([0])].tolist()

In [ ]:
len(indices)

In [ ]:
indices[:3]

In [ ]:
# json_path = home + '/llama_synth_function_output.json'

# with open(json_path, 'r') as j:
#     jfile = json.load(j)
#     samples = jfile

In [ ]:
for ind in indices:
    shape = samples[ind]['shape']
    inst = samples[ind]['size']
    seq = samples[ind]['net_seq']
    rec_size = None
    is_rectu = fn.is_rectangle_unfilled(seq)
    is_rect = fn.is_rectangle(seq)
    if is_rect:
        rec_size = is_rect[1]
    elif is_rectu:
        rect_size = is_rectu[1]
    rec_squares.append([ind, inst, rec_size])



In [ ]:
rec_squares

In [ ]:
len(rec_squares)

In [ ]:
import csv
square_path = home + '/rec_squares.csv'

In [ ]:
with open(square_path, 'w', newline='') as cfile:
    writer = csv.writer(cfile)
    writer.writerows(rec_squares)

In [ ]:
indices = df.index[(df['Shape'].isin(['square']))&(df['Shape_corr'] == 1)].tolist()

In [ ]:
indices